In [1]:
import numpy as np
import pandas as pd

In [2]:
match_results = pd.read_csv('data/original/results.csv')
match_results['date'] =  pd.to_datetime(match_results['date'], format='%Y-%m-%d')
match_results["year"] = match_results["date"].dt.year
match_results.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1875
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876


In [3]:
elo_ranking = pd.read_csv('data/generated/elo_ranking.csv', )
elo_ranking['date'] =  pd.to_datetime(elo_ranking['date'], format='%Y-%m-%d')
elo_ranking = elo_ranking.drop(['Unnamed: 0'], axis=1)
elo_ranking.head(5)

,team,elo,date
0,Scotland,1500.0,1872-11-30
1,England,1500.0,1872-11-30
2,Wales,1500.0,1872-11-30
3,Northern Ireland,1500.0,1872-11-30
4,United States,1500.0,1872-11-30


In [4]:
player_stats = pd.read_csv('data/generated/team_level_player_data.csv')
player_stats = player_stats.drop(['Unnamed: 0'], axis=1)
player_stats.head()

,nationality,year,overall_rating_max,overall_rating_min,overall_rating_avg,potential_max,potential_min,potential_avg,height,weight,...,Sprint_Speed,Reactions,Shot_Power,Stamina,Strength,Long_Shots,Aggression,Penalties,Marking,Standing_Tackle
0,Albania,2014,77,64,67.608696,80,66,71.434783,181.521739,76.260870,...,85.666667,72.2,77.2,79.4,79.0,66.8,82.4,70.666667,73.333333,70.2
1,Albania,2015,76,63,67.565217,78,63,71.695652,180.652174,75.739130,...,86.666667,73.6,77.8,79.9,78.8,66.0,84.4,72.333333,73.333333,71.8
2,Albania,2016,77,64,70.304348,82,66,72.434783,179.739130,75.391304,...,84.666667,74.8,79.4,81.8,77.5,69.8,85.6,73.333333,75.000000,76.0
3,Albania,2017,79,66,71.347826,83,68,74.304348,180.826087,75.043478,...,84.666667,74.8,76.4,83.2,75.6,68.3,83.0,69.666667,77.333333,77.8
4,Albania,2018,80,67,71.434783,89,69,73.478261,181.304348,75.521739,...,83.666667,74.0,78.2,82.6,76.4,69.7,84.6,69.333333,75.000000,75.6


In [5]:
goal_history = pd.read_csv('data/generated/goal_history_data.csv')
goal_history['date'] =  pd.to_datetime(goal_history['date'], format='%Y-%m-%d')
goal_history = goal_history.drop(['Unnamed: 0'], axis=1)
goal_history.head()

,away_goal_mean,away_goals_with_home,away_team,date,goal_diff_with_away,home_goal_mean,home_goals_with_away,home_team
0,0.000000,0.000000,England,1872-11-30,0,0.000000,0.000000,Scotland
1,0.000000,0.000000,Scotland,1873-03-08,0,0.000000,0.000000,England
2,2.000000,2.000000,England,1874-03-07,-2,1.000000,1.000000,Scotland
3,1.333333,1.333333,Scotland,1875-03-06,1,1.666667,1.666667,England
4,1.750000,1.750000,England,1876-03-04,-1,1.500000,1.500000,Scotland


In [6]:
dataset = pd.merge(match_results, elo_ranking,  how='left', left_on=['date','home_team'], right_on = ['date','team'])
dataset = dataset.drop(['team'], axis=1)
dataset.rename(columns = {"elo": "home_elo"}, inplace=True)

dataset = pd.merge(dataset, elo_ranking,  how='left', left_on=['date','away_team'], right_on = ['date','team'])
dataset = dataset.drop(['team'], axis=1)
dataset.rename(columns = {"elo": "away_elo"}, inplace=True)
dataset.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,home_elo,away_elo
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,1500.0,1500.0
1,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,1500.0,1500.0
2,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,1500.0,1500.0
3,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,1500.0,1500.0
4,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873,1522.5,1477.5


In [7]:
dataset_w = dataset.shape[1]
dataset = pd.merge(dataset, player_stats,  how='left', left_on=['year','home_team'], right_on = ['year','nationality'])
new_names = [(i, 'home_' + i) for i in dataset.iloc[:, dataset_w:].columns.values]
dataset.rename(columns = dict(new_names), inplace=True)
dataset = dataset.drop(['home_nationality'], axis=1)

dataset_w = dataset.shape[1]
dataset = pd.merge(dataset, player_stats,  how='left', left_on=['year','away_team'], right_on = ['year','nationality'])
new_names = [(i, 'away_' + i) for i in dataset.iloc[:, dataset_w:].columns.values]
dataset.rename(columns = dict(new_names), inplace=True)
dataset = dataset.drop(['away_nationality'], axis=1)

dataset.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,away_Sprint_Speed,away_Reactions,away_Shot_Power,away_Stamina,away_Strength,away_Long_Shots,away_Aggression,away_Penalties,away_Marking,away_Standing_Tackle
39238,2018-06-04,Italy,Netherlands,1,1,Friendly,Turin,Italy,False,2018,...,89.333333,84.4,84.4,84.500000,83.100000,79.100000,80.8,84.333333,83.000000,83.4
39239,2018-06-04,Serbia,Chile,0,1,Friendly,Graz,Austria,True,2018,...,81.333333,83.0,82.8,81.500000,77.800000,79.500000,85.0,82.000000,76.333333,80.8
39240,2018-06-04,Slovakia,Morocco,1,2,Friendly,Geneva,Switzerland,True,2018,...,80.666667,79.6,81.2,78.000000,77.000000,75.700000,80.8,79.000000,81.333333,78.4
39241,2018-06-04,Armenia,Moldova,0,0,Friendly,Kematen,Austria,True,2018,...,72.000000,68.4,67.4,61.087500,65.362500,42.975000,72.2,61.333333,73.333333,73.6
39242,2018-06-04,India,Kenya,3,0,Friendly,Mumbai,India,False,2018,...,81.000000,64.2,69.4,60.171429,67.757143,46.671429,73.2,57.666667,68.000000,63.2


In [8]:
dataset = pd.merge(dataset, goal_history,  how='left', left_on=['date','home_team'], right_on = ['date','home_team'])
dataset.rename(columns = {"home_team_x": "home_team", "away_team_x": "away_team"}, inplace=True)
dataset = dataset.drop(['away_team_y'], axis=1)
dataset.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,away_Long_Shots,away_Aggression,away_Penalties,away_Marking,away_Standing_Tackle,away_goal_mean,away_goals_with_home,goal_diff_with_away,home_goal_mean,home_goals_with_away
39350,2018-06-04,Italy,Netherlands,1,1,Friendly,Turin,Italy,False,2018,...,79.100000,80.8,84.333333,83.000000,83.4,1.914894,1.00,7,1.530612,1.35
39351,2018-06-04,Serbia,Chile,0,1,Friendly,Graz,Austria,True,2018,...,79.500000,85.0,82.000000,76.333333,80.8,1.574074,1.00,2,1.361111,3.00
39352,2018-06-04,Slovakia,Morocco,1,2,Friendly,Geneva,Switzerland,True,2018,...,75.700000,80.8,79.000000,81.333333,78.4,1.659574,2.00,-1,1.575000,1.00
39353,2018-06-04,Armenia,Moldova,0,0,Friendly,Kematen,Austria,True,2018,...,42.975000,72.2,61.333333,73.333333,73.6,0.486486,1.75,-2,1.250000,1.25
39354,2018-06-04,India,Kenya,3,0,Friendly,Mumbai,India,False,2018,...,46.671429,73.2,57.666667,68.000000,63.2,0.911111,0.00,0,1.870968,0.00


In [9]:
dataset.to_csv("data/generated/master_data.csv", index=False)